In [1]:
import torch
import torch.nn.functional as F
from PIL import Image
from open_clip import create_model_from_pretrained, get_tokenizer # works on open-clip-torch>=2.23.0, timm>=0.9.8
import os

In [2]:
model, preprocess = create_model_from_pretrained(model_name='ViT-SO400M-14-SigLIP-384', pretrained='ViT-SO400M-14-SigLIP-384/open_clip_pytorch_model.bin', cache_dir='ViT-SO400M-14-SigLIP-384', precision='fp16')  # precision='fp16' can slightly impact accuracy but halves vram (5.7G to 2.8G)
model.to('cuda')
model.eval()
if os.name != 'nt':
    model = torch.compile(model, backend="inductor")
tokenizer = get_tokenizer('ViT-SO400M-14-SigLIP-384')

In [3]:
def load_img(path):
    image = Image.open(path)
    image = preprocess(image).unsqueeze(0)
    image.type(torch.float16)
    image = image.to('cuda')
    return image

In [4]:
image1 = load_img('../grey rocket.jpg')
image2 = load_img('../green rocket.jpg')
image3 = load_img('../red blue and white light plane.jpg')
image_batched = torch.cat([image1, image2, image3], dim=0)

labels_list = ["grey missile"]
text = tokenizer(labels_list, context_length=model.context_length).to('cuda')

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image_batched)
    text_features = model.encode_text(text)
    image_features = F.normalize(image_features, dim=-1)
    text_features = F.normalize(text_features, dim=-1)
    text_similarity = torch.sigmoid(image_features @ text_features.T * model.logit_scale.exp() + model.logit_bias)
    best_match_bbox_idx = text_similarity.argmax(dim=0)

C:\Program Files\Python39\lib\site-packages\timm\models\vision_transformer.py:92: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  x = F.scaled_dot_product_attention(
